In [1]:
!pip install virtualenv

In [2]:
!pip uninstall -y qiskit qiskit-terra

Found existing installation: qiskit 1.2.4
Uninstalling qiskit-1.2.4:
  Successfully uninstalled qiskit-1.2.4
Found existing installation: qiskit-terra 0.46.0
Uninstalling qiskit-terra-0.46.0:
  Successfully uninstalled qiskit-terra-0.46.0


In [3]:
!pip install qiskit==0.46

  Using cached qiskit-0.46.0-py3-none-any.whl.metadata (12 kB)
  Using cached qiskit_terra-0.46.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached qiskit-0.46.0-py3-none-any.whl (9.6 kB)
Using cached qiskit_terra-0.46.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit-aer 0.15.1 requires qiskit>=1.1.0, but you have qiskit 0.46.0 which is incompatible.


In [4]:
!pip install qiskit-aer

  Using cached qiskit-1.2.4-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached qiskit-1.2.4-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Attempting uninstall: qiskit
    Found existing installation: qiskit 0.46.0
    Uninstalling qiskit-0.46.0:
      Successfully uninstalled qiskit-0.46.0


In [5]:
import qiskit
print(qiskit.__version__)

0.46.0


In [6]:
import traceback

In [7]:
from qiskit import execute, Aer

In [8]:
from qiskit import QuantumCircuit
import numpy as np
import random

def add_noise(circuit, p1, p2):
    # Add noise after each one-qubit and two-qubit gate
    noisy_circuit = circuit.copy()
    for gate in noisy_circuit.data:
        gate_type = gate[0].name
        qubits = gate[1]  # Correctly reference the qubits

        if gate_type in ['u', 'x', 'h', 'sx', 'id', 'rz']:  # One-qubit gates
            if random.random() < p1:
                # Apply a random Pauli gate (X, Y, or Z)
                noise_gate = random.choice(['x', 'y', 'z'])
                getattr(noisy_circuit, noise_gate)(qubits[0])  # Apply noise to the first qubit
        elif gate_type == 'cx':  # Two-qubit gate
            if random.random() < p2:
                noise_gate = random.choice(['x', 'y', 'z'])
                getattr(noisy_circuit, noise_gate)(qubits[0])  # Apply noise to the control qubit

    return noisy_circuit


In [9]:
def transform_to_gate_basis(circuit):
    basis_circuit = QuantumCircuit(circuit.num_qubits)

    for gate in circuit.data:
        gate_type = gate[0].name
        qubits = gate[1]  # Correctly reference the qubits

        if gate_type == 'cx':
            basis_circuit.cx(qubits[0], qubits[1])
        elif gate_type in ['h', 'sx']:
            basis_circuit.sx(qubits[0])
        elif gate_type == 'rz':
            basis_circuit.rz(gate[0].params[0], qubits[0])
        elif gate_type in ['u', 'id']:
            if gate_type == 'u':
                basis_circuit.x(qubits[0])  # Change this if you're applying specific parameters
            else:
                basis_circuit.id(qubits[0])
        # Handle additional gates as necessary

    return basis_circuit

In [10]:
from qiskit.circuit import QuantumCircuit

def qft(circuit, n):
    """ Quantum Fourier Transform """
    for j in range(n):
        for k in range(j):
            circuit.cp(np.pi/2**(j-k), k, j)
        circuit.h(j)

def quantum_sum(a, b, n):
    # Create a quantum circuit with enough qubits
    circuit = QuantumCircuit(n * 2)

    # Initialize inputs a and b
    for i in range(n):
        if (a >> i) & 1:
            circuit.x(i)
        if (b >> i) & 1:
            circuit.x(i + n)

    # Perform QFT on the first half
    qft(circuit, n)

    # Additional operations for the Draper adder would go here

    return circuit


In [12]:
from qiskit import QuantumCircuit
import numpy as np
import random

def add_noise(circuit, p1, p2):
    noisy_circuit = circuit.copy()
    for gate in noisy_circuit.data:
        gate_type = gate[0].name
        qubits = gate[1]

        if gate_type in ['u', 'x', 'h', 'sx', 'id', 'rz']:
            if random.random() < p1:
                noise_gate = random.choice(['x', 'y', 'z'])
                getattr(noisy_circuit, noise_gate)(qubits[0])
        elif gate_type == 'cx':
            if random.random() < p2:
                noise_gate = random.choice(['x', 'y', 'z'])
                getattr(noisy_circuit, noise_gate)(qubits[0])

    return noisy_circuit

def transform_to_gate_basis(circuit):
    basis_circuit = QuantumCircuit(circuit.num_qubits)

    for gate in circuit.data:
        gate_type = gate[0].name
        qubits = gate[1]

        if gate_type == 'cx':
            basis_circuit.cx(qubits[0], qubits[1])
        elif gate_type in ['h', 'sx']:
            basis_circuit.sx(qubits[0])
        elif gate_type == 'rz':
            basis_circuit.rz(gate[0].params[0], qubits[0])
        elif gate_type in ['u', 'id']:
            if gate_type == 'u':
                basis_circuit.x(qubits[0])
            else:
                basis_circuit.id(qubits[0])

    return basis_circuit

def qft(circuit, n):
    for j in range(n):
        for k in range(j):
            circuit.cp(np.pi / 2**(j - k), k, j)
        circuit.h(j)

def quantum_sum(a, b, n):
    circuit = QuantumCircuit(n * 2)

    for i in range(n):
        if (a >> i) & 1:
            circuit.x(i)
        if (b >> i) & 1:
            circuit.x(i + n)

    qft(circuit, n)

    return circuit

def analyze_noise(a, b, n, p1, p2):
    original_circuit = quantum_sum(a, b, n)
    transformed_circuit = transform_to_gate_basis(original_circuit)

    noisy_circuit = add_noise(transformed_circuit, p1, p2)

    # Instead of simulation, we will print the circuit and count operations
    print("Original Circuit:")
    print(original_circuit)

    print("\nTransformed Circuit:")
    print(transformed_circuit)

    print("\nNoisy Circuit:")
    print(noisy_circuit)

    # Analyze the gates in the noisy circuit
    gate_counts = noisy_circuit.count_ops()
    print("\nGate Counts in Noisy Circuit:", gate_counts)

    # Example of what could be the expected state (in a real case, we'd calculate this)
    expected_output = f"Expected output for {a} + {b} with noise levels p1={p1} and p2={p2}"
    print("\n" + expected_output)

# Example usage
analyze_noise(3, 5, 4, 0.1, 0.2)


Original Circuit:
     ┌───┐┌───┐                                                                »
q_0: ┤ X ├┤ H ├─■─────────────■─────────────────■──────────────────────────────»
     ├───┤└───┘ │P(π/2) ┌───┐ │                 │                              »
q_1: ┤ X ├──────■───────┤ H ├─┼────────■────────┼─────────────■────────────────»
     └───┘              └───┘ │P(π/4)  │P(π/2)  │       ┌───┐ │                »
q_2: ─────────────────────────■────────■────────┼───────┤ H ├─┼────────■───────»
                                                │P(π/8) └───┘ │P(π/4)  │P(π/2) »
q_3: ───────────────────────────────────────────■─────────────■────────■───────»
     ┌───┐                                                                     »
q_4: ┤ X ├─────────────────────────────────────────────────────────────────────»
     └───┘                                                                     »
q_5: ──────────────────────────────────────────────────────────────────────────»
     ┌───┐

### **Draper Adder Circuit Test**

In [13]:
from qiskit import QuantumCircuit
import numpy as np
import random

def add_noise(circuit, p1, p2):
    noisy_circuit = circuit.copy()
    for gate in noisy_circuit.data:
        gate_type = gate[0].name
        qubits = gate[1]

        if gate_type in ['u', 'x', 'h', 'sx', 'id', 'rz']:
            if random.random() < p1:
                noise_gate = random.choice(['x', 'y', 'z'])
                getattr(noisy_circuit, noise_gate)(qubits[0])
        elif gate_type == 'cx':
            if random.random() < p2:
                noise_gate = random.choice(['x', 'y', 'z'])
                getattr(noisy_circuit, noise_gate)(qubits[0])

    return noisy_circuit

def transform_to_gate_basis(circuit):
    basis_circuit = QuantumCircuit(circuit.num_qubits)

    for gate in circuit.data:
        gate_type = gate[0].name
        qubits = gate[1]

        if gate_type == 'cx':
            basis_circuit.cx(qubits[0], qubits[1])
        elif gate_type in ['h', 'sx']:
            basis_circuit.sx(qubits[0])
        elif gate_type == 'rz':
            basis_circuit.rz(gate[0].params[0], qubits[0])
        elif gate_type in ['u', 'id']:
            if gate_type == 'u':
                basis_circuit.x(qubits[0])
            else:
                basis_circuit.id(qubits[0])

    return basis_circuit

def qft(circuit, n):
    """Quantum Fourier Transform"""
    for j in range(n):
        for k in range(j):
            circuit.cp(np.pi / 2**(j - k), k, j)
        circuit.h(j)

def inverse_qft(circuit, n):
    """Inverse Quantum Fourier Transform"""
    for j in range(n):
        circuit.h(j)
        for k in range(j):
            circuit.cp(-np.pi / 2**(j - k), k, j)

def quantum_sum(a, b, n):
    circuit = QuantumCircuit(n * 2 + 1)  # +1 for carry bit

    # Initialize input a
    for i in range(n):
        if (a >> i) & 1:
            circuit.x(i)

    # Initialize input b
    for i in range(n):
        if (b >> i) & 1:
            circuit.x(i + n)

    # QFT on first n qubits (a)
    qft(circuit, n)

    # Add b to the first n qubits using controlled rotations
    for i in range(n):
        circuit.cx(n + i, i)

    # Inverse QFT
    inverse_qft(circuit, n)

    return circuit

def analyze_noise(a, b, n, p1, p2):
    original_circuit = quantum_sum(a, b, n)
    transformed_circuit = transform_to_gate_basis(original_circuit)

    noisy_circuit = add_noise(transformed_circuit, p1, p2)

    # Print circuit details
    print("Original Circuit:")
    print(original_circuit)

    print("\nTransformed Circuit:")
    print(transformed_circuit)

    print("\nNoisy Circuit:")
    print(noisy_circuit)

    # Analyze the gates in the noisy circuit
    gate_counts = noisy_circuit.count_ops()
    print("\nGate Counts in Noisy Circuit:", gate_counts)

    # Example of what could be the expected state
    expected_output = f"Expected output for {a} + {b} with noise levels p1={p1} and p2={p2}"
    print("\n" + expected_output)

# Example usage
analyze_noise(3, 5, 4, 0.1, 0.2)


Original Circuit:
     ┌───┐┌───┐                                                       ┌───┐»
q_0: ┤ X ├┤ H ├─■─────────────■─────────────────■─────────────────────┤ X ├»
     ├───┤└───┘ │P(π/2) ┌───┐ │                 │                     └─┬─┘»
q_1: ┤ X ├──────■───────┤ H ├─┼────────■────────┼─────────────■─────────┼──»
     └───┘              └───┘ │P(π/4)  │P(π/2)  │       ┌───┐ │         │  »
q_2: ─────────────────────────■────────■────────┼───────┤ H ├─┼─────────┼──»
                                                │P(π/8) └───┘ │P(π/4)   │  »
q_3: ───────────────────────────────────────────■─────────────■─────────┼──»
     ┌───┐                                                              │  »
q_4: ┤ X ├──────────────────────────────────────────────────────────────■──»
     └───┘                                                                 »
q_5: ──────────────────────────────────────────────────────────────────────»
     ┌───┐                                                

In [14]:
# Run multiple experiments with varying noise levels
def analyze_multiple_noises(a, b, n, noise_levels):
    for p1, p2 in noise_levels:
        print(f"\nAnalyzing with noise levels p1={p1}, p2={p2}")
        analyze_noise(a, b, n, p1, p2)

# Example noise levels to test
noise_levels = [
    (0.01, 0.01),
    (0.05, 0.05),
    (0.1, 0.1),
    (0.2, 0.2),
    (0.3, 0.3)
]

# Run analysis for two numbers 3 and 5 with 4 qubits
analyze_multiple_noises(3, 5, 4, noise_levels)


Analyzing with noise levels p1=0.01, p2=0.01
Original Circuit:
     ┌───┐┌───┐                                                       ┌───┐»
q_0: ┤ X ├┤ H ├─■─────────────■─────────────────■─────────────────────┤ X ├»
     ├───┤└───┘ │P(π/2) ┌───┐ │                 │                     └─┬─┘»
q_1: ┤ X ├──────■───────┤ H ├─┼────────■────────┼─────────────■─────────┼──»
     └───┘              └───┘ │P(π/4)  │P(π/2)  │       ┌───┐ │         │  »
q_2: ─────────────────────────■────────■────────┼───────┤ H ├─┼─────────┼──»
                                                │P(π/8) └───┘ │P(π/4)   │  »
q_3: ───────────────────────────────────────────■─────────────■─────────┼──»
     ┌───┐                                                              │  »
q_4: ┤ X ├──────────────────────────────────────────────────────────────■──»
     └───┘                                                                 »
q_5: ──────────────────────────────────────────────────────────────────────»
     ┌───┐  

####**1. How Does the Noise Affect the Results?**

Noise in quantum computing can manifest in various ways, impacting the accuracy and reliability of quantum operations. Here's how noise affects the results:

Error Rates: As noise levels increase (both
𝑝
1
p1 and
𝑝
2
p2), the probability of applying incorrect gates increases. This can lead to:

Bit Flips: Random application of Pauli gates (X, Y, Z) can change the intended state of qubits, causing incorrect results.
Circuit Failures: With higher noise, the probability of significant errors increases, potentially causing the entire circuit to produce a completely wrong output.
Output Variability: The results may become less deterministic, showing a distribution of possible outcomes rather than a single expected result. For example, if you're trying to add 3 and 5, higher noise levels may result in outputs like 6, 7, or even 1, depending on how the noise affects the circuit.

Circuit Depth Impact: In circuits with many gates, noise accumulates, leading to a more pronounced effect on the final outcome. The more gates present, the greater the chance for noise to introduce errors, which can drastically alter results.

####**2. Is There a Way to Decrease the Effect of Noise?**

There are several strategies to mitigate the effects of noise in quantum circuits:

Quantum Error Correction: This is a method specifically designed to detect and correct errors without directly measuring the quantum state. Techniques include:

Shor's Code: This encodes a single logical qubit into several physical qubits, allowing for the correction of errors that affect any one of the qubits.
Surface Codes: These are particularly promising for practical implementations, providing a way to correct errors by leveraging redundancy in qubit arrangements.
Noise-Resilient Circuit Design: Techniques for designing circuits that are less susceptible to noise include:

Gate Optimization: Using fewer gates or more stable gates reduces the chances of noise affecting the computation.
Circuit Restructuring: Analyzing circuit structure to minimize depth and qubit interactions can help mitigate error accumulation.
Feedback and Control: Implementing adaptive feedback systems allows for real-time adjustments based on detected errors during computation. This can help in correcting errors dynamically as they occur.

Using High-Fidelity Gates: Selecting quantum operations that have been shown to have lower error rates can improve overall circuit performance under noise.

####**3. How Does the Number of Gates Used Affect the Results?**

The number of gates in a quantum circuit has a direct relationship with the impact of noise on the computation:

Error Accumulation: Every additional gate introduces a new opportunity for noise to corrupt the operation:

Linear Increase in Error Probability: The likelihood of at least one gate being affected by noise increases with each additional gate. For example, if each gate has a 1% error rate, in a sequence of 100 gates, the cumulative probability of having at least one error becomes significant.
Circuit Depth: Longer circuits (higher gate counts) tend to experience greater fidelity loss. The depth of the circuit impacts how long qubits are held in a superposition state, which can be degraded by noise:

Increased Decoherence: Longer circuit depths may lead to qubits losing their quantum properties due to environmental interactions, further affecting results.
Complexity of Errors: More gates mean more potential interactions among qubits, which can lead to complex error patterns that are harder to correct or predict.

Trade-offs: Sometimes, a simpler circuit (fewer gates) might not perform the desired computation efficiently but will be more robust against noise. Finding the right balance between circuit complexity and error resilience is crucial for practical quantum computing.